In [10]:
#!pip install Pillow==10.1.0

In [11]:
#!pip install torch==2.1.2

In [12]:
#!pip install torchvision==0.16.2

In [13]:
#!pip install transformers==4.40.0

In [14]:
#!pip install sentencepiece==0.1.99

In [15]:
#!pip install accelerate==0.30.1

In [16]:
#!pip install bitsandbytes==0.43.1

In [17]:
#!git clone https://huggingface.co/openbmb/MiniCPM-Llama3-V-2_5-int4

In [1]:
# Load model, create function
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer
import time
import os

model = AutoModel.from_pretrained('MiniCPM-Llama3-V-2_5-int4', trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('MiniCPM-Llama3-V-2_5-int4', trust_remote_code=True)
model.eval()

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


MiniCPMV(
  (llm): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attentio

In [2]:
def scale_image(input_image_path, output_image_path, width=None):
    original_image = Image.open(input_image_path).convert('RGB')
    w, h = original_image.size
    if width is not None:
        # calculate the height using the same aspect ratio
        height = int(h * width / w)
        resized_img = original_image.resize((width, height))
    else:
        resized_img = original_image
    resized_img.save(output_image_path)

def ProcessImage(img, question):
    start_time = time.time()
    image = Image.open(img).convert('RGB')
    if not os.path.isfile('out-' + img):
        scale_image(img, 'out-' + img, width=800)
    image = Image.open('out-' + img).convert('RGB')
    msgs = [{'role': 'user', 'content': question}]
    res = model.chat(
        image=image,
        msgs=msgs,
        tokenizer=tokenizer,
        sampling=True,
        temperature=0.7
    )
    print(res)
    end_time = time.time()
    time_diff = end_time - start_time
    minutes, seconds = divmod(time_diff,60)
    print("[{0} min, {1} sec]".format(int(minutes), int(seconds)))

In [3]:
ProcessImage(
  "000sample.jpg",
  "convert this image to a json structure answer, summarise all content " +
  "that is either a store_name, date_time, total, and items with name, " +
  "quantity, and price, ignoring anything with names 'cash' or 'credit' " +
  "in the image it's a till slip with the store name which doesn't need " +
  "ownership details, the payment amount, and a json array with each " +
  "line item purchased with focus on name of item, the quantity received" +
  ", and the price of the listed item, while ignoring items that are " +
  "just additional business info as well, or have price '0.00'"
)

{
  "store_name": "KFC South Africa Copper Moon Trading As KFC Rosebank",
  "date_time": "Nov'01" 18:12:12,
  "total": 49.90,
  "items": [
    {
      "name": "1 Chicken LunchBox",
      "quantity": 1,
      "price": 49.90
    },
    {
      "name": "1 Small Lbox M1 S/SF",
      "quantity": 1,
      "price": 0.00
    }
  ]
}
[14 min, 44 sec]


In [3]:
ProcessImage(
    "cv-ernestl.png",
    "as a professional full stack developer, what are the main best points " +
    "in my resume, and then what full stack project(s) can I do to improve " +
    "my full-stack developer mindset, as well as broaden my knowledge in AI" +
    ", ML, and OCR, since I want to document all my knowledge in sample " +
    "projects in all of my skills on my github. This intends to broaden my " +
    "LinkedIn profile in the broadest way, and I can make each project into" +
    " a blog post"
)

The main points highlighted in your resume as a full-stack developer include:

1. Proficiency in various technologies such as MSSQL, MySQL, PHP, Python, and JavaScript.
2. Experience with software development, including data analysis, management, and implementation.
3. Familiarity with version control systems like Git and web development frameworks like Bootstrap.
4. Knowledge of machine learning, artificial intelligence, and optical character recognition (OCR) concepts.
5. A history of working on personal projects and contributions to open-source platforms.

To improve your full-stack developer mindset and broaden your knowledge in AI, ML, and OCR, you can consider working on projects that integrate these technologies. For instance, you could create a web application that uses OCR to recognize and extract information from scanned documents, or build an AI-powered recommendation system for products. Additionally, you could explore ML-based projects such as sentiment analysis on custome